<a href="https://colab.research.google.com/github/samikshagadhave/predictive-analysis-model-for-manufacturing-data/blob/main/predictive_analysis_model_for_manufacturing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Define parameters
num_records = 1000  # Number of rows
machine_ids = [f"Machine_{i}" for i in range(1, 11)]  # 10 machines
temperature_range = (60, 120)  # Temperature in degrees Celsius
runtime_range = (1, 24)  # Runtime in hours

# Generate data
data = {
    "Machine_ID": np.random.choice(machine_ids, size=num_records),
    "Temperature": np.random.uniform(*temperature_range, size=num_records).round(2),
    "Run_Time": np.random.uniform(*runtime_range, size=num_records).round(1),
    "Downtime_Flag": np.random.choice([0, 1], size=num_records, p=[0.8, 0.2]),  # 20% downtime
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the first few rows
print(df.head())

# Save to CSV (optional)
df.to_csv("synthetic_machine_data.csv", index=False)


  Machine_ID  Temperature  Run_Time  Downtime_Flag
0  Machine_7        75.36       2.1              0
1  Machine_4       103.57       1.9              1
2  Machine_8        95.58      18.2              0
3  Machine_5        66.13      16.2              0
4  Machine_7       115.13      19.4              0


In [ ]:
!pip install flask pandas scikit-learn joblib

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://lnoi0qarn48-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify
#from flask_ngrok import run_with_ngrok
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import joblib
import os

# Generate synthetic dataset
def generate_data(num_records=1000):
    np.random.seed(42)
    machine_ids = [f"Machine_{i}" for i in range(1, 11)]
    temperature_range = (60, 120)
    runtime_range = (1, 24)
    data = {
        "Machine_ID": np.random.choice(machine_ids, size=num_records),
        "Temperature": np.random.uniform(*temperature_range, size=num_records).round(2),
        "Run_Time": np.random.uniform(*runtime_range, size=num_records).round(1),
        "Downtime_Flag": np.random.choice([0, 1], size=num_records, p=[0.8, 0.2]),
    }
    return pd.DataFrame(data)

# Initialize Flask app
app = Flask(__name__)
#run_with_ngrok(app)

# Global variables to store dataset and model
dataset = None
model = None

# Upload endpoint
@app.route('/upload', methods=['POST'])
def upload_file():
    global dataset
    file = request.files['file']

    if not file:
        return jsonify({"error": "No file provided"}), 400

    try:
        dataset = pd.read_csv(file)
        return jsonify({"message": "File uploaded successfully", "columns": list(dataset.columns)}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 400

# Train endpoint
@app.route('/train', methods=['POST'])
def train_model():
    global dataset, model

    if dataset is None:
        return jsonify({"error": "No dataset uploaded. Use /upload to upload a dataset."}), 400

    required_columns = {"Temperature", "Run_Time", "Downtime_Flag"}
    if not required_columns.issubset(dataset.columns):
        return jsonify({"error": f"Dataset must contain the following columns: {required_columns}"}), 400

    try:
        X = dataset[["Temperature", "Run_Time"]]
        y = dataset["Downtime_Flag"]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = LogisticRegression()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        joblib.dump(model, "model.pkl")

        return jsonify({"message": "Model trained successfully", "accuracy": accuracy, "f1_score": f1}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 400

# Predict endpoint
@app.route('/predict', methods=['POST'])
def predict():
    global model

    if model is None:
        return jsonify({"error": "No model trained. Use /train to train a model."}), 400

    try:
        data = request.get_json()
        temperature = data.get("Temperature")
        run_time = data.get("Run_Time")

        if temperature is None or run_time is None:
            return jsonify({"error": "Temperature and Run_Time are required fields"}), 400

        input_data = np.array([[temperature, run_time]])
        prediction = model.predict(input_data)[0]
        confidence = max(model.predict_proba(input_data)[0])

        downtime = "Yes" if prediction == 1 else "No"

        return jsonify({"Downtime": downtime, "Confidence": round(confidence, 2)}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 400

if __name__ == "__main__":
    # Generate and save synthetic data (optional)
    df = generate_data()
    df.to_csv("synthetic_machine_data.csv", index=False)
    print("Synthetic dataset saved as 'synthetic_machine_data.csv'.")
    app.run()


Synthetic dataset saved as 'synthetic_machine_data.csv'.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
